In [1]:
import numpy as np
import pandas as pd
#import psycopg2
#import sqlalchemy
import matplotlib as plt

%matplotlib inline

In [20]:
FullUsers = pd.read_csv('/home/deth/FullUsers.csv')
Friends = pd.read_csv('/home/deth/Friends.csv')
Posts = pd.read_csv('/home/deth/PostsHistory.csv')
UsersGroups = pd.read_csv('/home/deth/UsersGroups.csv')
Likes = pd.read_csv('/home/deth/Likes.csv')

In [28]:
id_dict = {}
i = 0
for index, row in FullUsers.iterrows():
    id_dict[row['uid']] = index
    i += 1

In [32]:
UsersGroups.max()

1000

In [22]:
UsersGroups.drop(columns=['Unnamed: 0'], inplace = True)

FullUsers['number_of_groups'] = 0
UsersGroups = UsersGroups.groupby('uid')['gid'].count()

for i, v in UsersGroups.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_groups'] = v
        
FullUsers[['number_of_groups']].fillna(value=0, inplace=True)

/home/deth/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [23]:
FullUsers['number_of_friends'] = 0

Friends.drop(columns=['Unnamed: 0'], inplace = True)

Friends = Friends.groupby('uid1')['uid2'].count()

for i, v in Friends.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_friends'] = v
        
FullUsers[['number_of_friends']].fillna(value=0, inplace=True)

In [24]:
#Posts.drop(columns=['Unnamed: 0'], inplace = True)

FullUsers['number_of_original_posts'] = 0

PostsOriginal = Posts[(Posts.from_id == Posts.owner_id) & Posts.original].groupby('from_id')

NumberOfOriginalPosts = PostsOriginal.original.count()

for i, v in NumberOfOriginalPosts.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_original_posts'] = v
        


NumberOfOriginalPostsLikes = PostsOriginal.likes.sum()

FullUsers['number_of_original_posts_likes'] = 0

for i, v in NumberOfOriginalPostsLikes.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_original_posts_likes'] = v
        
FullUsers[['number_of_original_posts', 'number_of_original_posts_likes']].fillna(value=0, inplace=True)

In [39]:
NumberOfOriginalPosts[305944914]

18

In [25]:
PostsUnoriginal = Posts[(Posts.from_id == Posts.owner_id) & (Posts.original == False)].groupby('from_id')

FullUsers['number_of_unoriginal_posts'] = 0

NumberOfUnoriginalPosts = PostsUnoriginal.original.count()

for i, v in NumberOfUnoriginalPosts.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_unoriginal_posts'] = v
 

FullUsers['number_of_unoriginal_posts_likes'] = 0

NumberOfUnoriginalPostsLikes = PostsUnoriginal.likes.sum()

for i, v in NumberOfUnoriginalPostsLikes.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_unoriginal_posts_likes'] = v
        
FullUsers[['number_of_unoriginal_posts', 'number_of_unoriginal_posts_likes']].fillna(value=0, inplace=True)

In [26]:
FullUsers['number_of_own_posts_comments'] = 0
NumberOfOwnPostsComments = Posts[Posts.from_id == Posts.owner_id].groupby('from_id').comments.sum()
for i, v in NumberOfOwnPostsComments.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_own_posts_comments'] = v
        
FullUsers[['number_of_own_posts_comments']].fillna(value=0, inplace=True)

In [27]:
FullUsers['number_of_selflikes'] = 0

Likes.drop(columns=['Unnamed: 0'], inplace = True)

NumberOfSelflikes = Likes[Likes.from_id == Likes.owner_id].groupby('from_id').owner_id.count()
for i, v in NumberOfSelflikes.items():
    if i in id_dict:
        FullUsers.at[id_dict[i],'number_of_selflikes'] = v
        
FullUsers[['number_of_selflikes']].fillna(value=0, inplace=True)

In [19]:
#FullUsers.sort_values(['number_of_groups', 'number_of_friends', 'number_of_original_posts', 'number_of_unoriginal_posts',
#                      'number_of_original_posts_likes', 'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments'], ascending=[False, False, False, False, False, False, False], inplace=True)

In [42]:
print((FullUsers.number_of_original_posts + FullUsers.number_of_unoriginal_posts).mean(), ' ', (FullUsers.number_of_original_posts.max()))

372.22794600990403   130556


In [43]:
Full = FullUsers[(FullUsers['number_of_original_posts']>0) | (FullUsers['number_of_unoriginal_posts']>0)].copy()

In [44]:
print((Full.number_of_original_posts + Full.number_of_unoriginal_posts).mean(), ' ', (Full.number_of_original_posts.max()))

638.4447910402886   130556


In [47]:
print(Full.shape)

(26608, 27)


In [54]:
Full['number_of_posts'] = Full.number_of_original_posts + Full.number_of_unoriginal_posts

In [56]:
Full.sort_values(['number_of_posts'], ascending=[False], inplace=True)

In [57]:
MightBeLeaders = Full[:5322].copy()

In [67]:
FullUsers['number_of_posts'] = FullUsers['number_of_original_posts']+FullUsers['number_of_unoriginal_posts']
FullUsers['number_of_likes'] = FullUsers['number_of_original_posts_likes']+FullUsers['number_of_unoriginal_posts_likes']

In [69]:
print(FullUsers.number_of_posts.mean())
print(FullUsers.number_of_likes.mean())
print(FullUsers.number_of_groups.mean())
print(FullUsers.number_of_friends.mean())

372.22794600990403
2211.6938516148825
85.67947762829222
400.1491081993076


In [75]:
LeadersPosts = FullUsers.sort_values(['number_of_posts'], ascending=[False])[:9000]
LeadersLikes = FullUsers.sort_values(['number_of_likes'], ascending=[False])[:9000]
LeadersGroups = FullUsers.sort_values(['number_of_groups'], ascending=[False])[:9000]
LeadersFriends = FullUsers.sort_values(['number_of_friends'], ascending=[False])[:9000]


In [84]:
PL = pd.merge(LeadersPosts, LeadersLikes, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])
PLG = pd.merge(PL, LeadersGroups, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])
PLGF = pd.merge(PLG, LeadersFriends, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])

In [85]:
PLGF

,uid,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,...,number_of_groups,number_of_friends,number_of_original_posts,number_of_original_posts_likes,number_of_unoriginal_posts,number_of_unoriginal_posts_likes,number_of_own_posts_comments,number_of_selflikes,number_of_posts,number_of_likes
0,41451787,Георг ...,Опря,2,дж†Динамит†Среди друзей Джордж,id41451787,17.11.1992,1,1,https://sun1-27.userapi.com/c849228/v849228511...,...,1000,1315,55491,58803,122159,71116,0,217,177650,129919
1,115934753,Елена ...,Влади,1,NaN,bomond97,5.9,1,1,https://sun1-18.userapi.com/c844418/v844418795...,...,1000,4425,7086,26979,62252,149735,0,14640,69338,176714
2,1113834,Инна ...,Соснина,1,NaN,sosninainna,29.1.1987,1,1,https://sun1-21.userapi.com/c846420/v846420913...,...,1000,4069,2334,20319,60195,149418,31096,3482,62529,169737
3,22464473,Мария ...,Павленко,1,NaN,id22464473,26.5.1991,1,1,https://sun1-88.userapi.com/c852024/v852024172...,...,1000,2781,5611,16348,53338,24530,0,8531,58949,40878
4,7733695,Артак ...,Григорян,2,NaN,artak_grigoryan,11.5.1980,1,1,https://pp.userapi.com/VQMq1gR_Vpc1h4vANf-sxzV...,...,590,3694,6745,53937,49526,183733,18352,4058,56271,237670
5,54498403,Дмитрий ...,Круглов,2,NaN,dmitriy_kruglov1,30.6,143,1,https://sun1-17.userapi.com/c846419/v846419169...,...,1000,4830,17832,229462,33605,190312,0,1614,51437,419774
6,2818498,Александр ...,Николаевич,2,NaN,id2818498,19.5.1969,2,1,https://pp.userapi.com/bg05FCIsNeoyVbhlPE8jo9q...,...,1000,3842,14517,28202,28628,41531,655,38876,43145,69733
7,255430049,Оксана ...,Мальчикова,1,NaN,o.malchikova76,11.12.1976,4161,1,https://pp.userapi.com/c844618/v844618055/1faf...,...,1000,646,1640,1807,40637,16927,0,35,42277,18734
8,150736003,Людмила ...,Тимчук,1,NaN,idbelvederska33,12.6,0,2,https://pp.userapi.com/HJn0OtwXpDLLBYVklQHle1m...,...,274,2506,691,4859,41442,135737,0,1105,42133,140596
9,114047500,Виктория ...,Лежнева,1,NaN,swag_baby_eah,22.9.1993,223,2,https://sun1-21.userapi.com/c850432/v850432618...,...,1000,1018,11455,17290,26348,28780,0,370,37803,46070


In [86]:
print(PLGF.number_of_posts.min(), "    ", FullUsers.number_of_posts.mean())
print(PLGF.number_of_likes.min(), "    ", FullUsers.number_of_likes.mean())
print(PLGF.number_of_groups.min(), "    ", FullUsers.number_of_groups.mean())
print(PLGF.number_of_friends.min(), "    ", FullUsers.number_of_friends.mean())

341      372.22794600990403
1352      2211.6938516148825
119      85.67947762829222
482      400.1491081993076


In [80]:
PLGF.number_of_posts_y.min()

number_of_posts_y    341
number_of_posts_y    341
dtype: int64

In [83]:
FullUsers.columns

Index(['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'],
      dtype='object')

In [87]:
print(LeadersPosts.number_of_posts.min(), "   ", FullUsers.number_of_posts.mean())

341     372.22794600990403


Another f*ing try

In [88]:
LeadersPosts2 = FullUsers.sort_values(['number_of_posts'], ascending=[False])[FullUsers.number_of_posts/FullUsers.number_of_posts.mean() > 1.2]
LeadersLikes2 = FullUsers.sort_values(['number_of_likes'], ascending=[False])[FullUsers.number_of_likes/FullUsers.number_of_likes.mean() > 1.2]
LeadersGroups2 = FullUsers.sort_values(['number_of_groups'], ascending=[False])[FullUsers.number_of_groups/FullUsers.number_of_groups.mean() > 1.2]
LeadersFriends2 = FullUsers.sort_values(['number_of_friends'], ascending=[False])[FullUsers.number_of_friends/FullUsers.number_of_friends.mean() > 1.2]

/home/deth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.
/home/deth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/home/deth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until
/home/deth/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [90]:
PL2 = pd.merge(LeadersPosts, LeadersLikes2, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])
PLG2 = pd.merge(PL, LeadersGroups2, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])
PLGF2 = pd.merge(PLG, LeadersFriends2, how='inner', on=['uid', 'first_name', 'last_name', 'sex', 'nickname', 'domain', 'bdate',
       'city', 'country', 'photo_50', 'photo_100', 'photo_200_orig',
       'can_write_private_message', 'status', 'data_loaded', 'mobile_phone',
       'home_phone', 'group_list_loaded', 'comments_loaded',
       'number_of_groups', 'number_of_friends', 'number_of_original_posts',
       'number_of_original_posts_likes', 'number_of_unoriginal_posts',
       'number_of_unoriginal_posts_likes', 'number_of_own_posts_comments',
       'number_of_selflikes', 'number_of_posts', 'number_of_likes'])

In [93]:
print(PLGF2.number_of_posts.min(), "    ", FullUsers.number_of_posts.mean())
print(PLGF2.number_of_likes.min(), "    ", FullUsers.number_of_likes.mean())
print(PLGF2.number_of_groups.min(), "    ", FullUsers.number_of_groups.mean())
print(PLGF2.number_of_friends.min(), "    ", FullUsers.number_of_friends.mean())

341      372.22794600990403
1352      2211.6938516148825
119      85.67947762829222
481      400.1491081993076


In [95]:
a = PLGF2.number_of_groups.min()/FullUsers.number_of_groups.mean()
b = PLGF2.number_of_friends.min()/ FullUsers.number_of_friends.mean()
print(a, " ", b)

1.3888973566839886   1.202051910510374


In [118]:
df = FullUsers[(FullUsers.number_of_groups/FullUsers.number_of_groups.mean()>1) & (FullUsers.number_of_friends/ FullUsers.number_of_friends.mean()>1) &
              (FullUsers.number_of_posts>0)].copy()

In [119]:
df.shape

(3950, 29)

In [120]:
df

,uid,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,...,number_of_groups,number_of_friends,number_of_original_posts,number_of_original_posts_likes,number_of_unoriginal_posts,number_of_unoriginal_posts_likes,number_of_own_posts_comments,number_of_selflikes,number_of_posts,number_of_likes
17,355056237,Светлана ...,Аминова,1,NaN,agat068,NaN,60,1,https://pp.userapi.com/c847121/v847121673/108f...,...,295,4571,4,19,41,55,4,0,45,74
19,355464403,Repost ...,Dobro,1,NaN,repost_dobro,NaN,49,1,https://pp.userapi.com/c837529/v837529607/4546...,...,275,1155,581,4924,254,1276,37,24,835,6200
33,434130018,Сергей ...,Чжоу,2,NaN,czhou,NaN,1450,97,https://pp.userapi.com/c834301/v834301885/16f8...,...,114,2692,46,395,1,2,9,0,47,397
37,416521051,Танечка ...,Царевская,1,NaN,id416521051,16.12.1974,149,1,https://pp.userapi.com/c836436/v836436051/42f1...,...,894,447,31,110,4034,837,42,44,4065,947
45,460663791,Екатерина ...,Соколова,1,NaN,esokolova2018,4.12.1994,4644,1,https://sun1-24.userapi.com/c854532/v854532849...,...,203,2153,2794,1983,17,97,1057,3,2811,2080
54,371868001,Вера ...,Пронина,1,NaN,id371868001,15.11,60,1,https://sun9-1.userapi.com/c840120/v840120965/...,...,180,485,25,46,615,130,3,8,640,176
82,144170125,Ольчик ...,Борисова,1,NaN,ochirikova,3.2,126,1,https://pp.userapi.com/c845220/v845220913/1a59...,...,86,429,8,172,231,1408,22,6,239,1580
135,5624523,Людмила ...,Малей,1,Люська,id5624523,11.9.1984,95,1,https://pp.userapi.com/AtlKejwE42ocjZq7Y6te6RS...,...,839,635,550,2864,668,1853,877,255,1218,4717
149,26051368,Анна ...,Казакова,1,NaN,id26051368,23.8.1991,146,1,https://sun1-83.userapi.com/c627326/v627326368...,...,287,502,250,8120,1232,5242,0,102,1482,13362
208,220261934,Объемные-Буквы ...,И-Слова,1,NaN,id220261934,NaN,72,1,https://pp.userapi.com/c628830/v628830934/3979...,...,170,584,54,49,7,10,67,1,61,59


In [151]:
Transaction = df[(df['number_of_selflikes']/df['number_of_selflikes'].mean()-1>0.3)].copy()
Transaction.shape

(666, 29)

In [152]:
Slave = df[(df['number_of_selflikes']/df['number_of_selflikes'].mean()-1<0.3) & ((df['number_of_likes']/df['number_of_likes'].mean()-1)>0)].copy()
Slave.shape

(524, 29)

In [154]:
df.sort_values(['number_of_selflikes'], ascending=[False])

,uid,first_name,last_name,sex,nickname,domain,bdate,city,country,photo_50,...,number_of_groups,number_of_friends,number_of_original_posts,number_of_original_posts_likes,number_of_unoriginal_posts,number_of_unoriginal_posts_likes,number_of_own_posts_comments,number_of_selflikes,number_of_posts,number_of_likes
36926,2818498,Александр ...,Николаевич,2,NaN,id2818498,19.5.1969,2,1,https://pp.userapi.com/bg05FCIsNeoyVbhlPE8jo9q...,...,1000,3842,14517,28202,28628,41531,655,38876,43145,69733
30934,43342479,Анна ...,Сурис,1,NaN,id43342479,19.2.1956,2,1,https://sun1-86.userapi.com/c852224/v852224393...,...,113,694,11296,20290,60733,86509,8597,29971,72029,106799
41987,1162323,Денис ...,Богданов,2,Macromusic,id1162323,5.1.1987,2,1,https://sun1-83.userapi.com/c845416/v845416862...,...,1000,4532,846,18857,30754,164721,0,28378,31600,183578
21155,115934753,Елена ...,Влади,1,NaN,bomond97,5.9,1,1,https://sun1-18.userapi.com/c844418/v844418795...,...,1000,4425,7086,26979,62252,149735,0,14640,69338,176714
37265,383065,Илья ...,Шпиркин,2,NaN,id383065,3.5,0,1,https://pp.userapi.com/c935/v935065/24/O3j_ng6...,...,1000,3938,5654,13350,18748,17622,557,13972,24402,30972
42981,134469223,Светлана ...,Кис,1,NaN,id134469223,NaN,2,1,https://pp.userapi.com/c638624/v638624084/58d9...,...,1000,1948,434,2330,19892,76150,258,12802,20326,78480
30715,40558029,Ирина ...,Киселева,1,Вячеславовна,id40558029,13.5,0,0,https://vk.com/images/camera_50.png?ava=1,...,502,867,394,240,12992,15620,0,12595,13386,15860
40546,5791946,Вероника ...,Лапина,1,NaN,byy11,NaN,0,9,https://pp.userapi.com/c830609/v830609553/b132...,...,359,3876,3471,1044,18199,18646,0,11376,21670,19690
26870,73707001,Мария ...,Соколова,1,[RDJ + GAGA + MJ forever],olovooo,13.1.1999,1,1,https://sun1-26.userapi.com/c831508/v831508580...,...,339,4113,494,11527,10521,16410,0,9162,11015,27937
23039,195798,Леночка ...,Щекочихина,1,NaN,elene,17.1.1987,1,1,https://pp.userapi.com/c850620/v850620080/f16d...,...,320,443,740,9058,31728,29383,0,8735,32468,38441


In [155]:
df.to_csv(r'/home/deth/df.csv')